<a href="https://colab.research.google.com/github/yul77/Review/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4_%EC%B2%98%EB%A6%AC_1%EC%9E%A5_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 자연어 처리 1장

**각종 설정값 정하기**

In [ ]:
#pip 버전 다운그레이드
!pip install pip==23.0


# ratsnlp 설치하려는데 계속 에러나서 pip자체를 다운그레이드 시도

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
#호환 가능한 버전으로 ratsnlp와 torch를 설치
!pip install torch==2.4.0
!pip install ratsnlp

In [ ]:
#설정값 선언

from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="nsmc",
    downstream_corpus_root_dir="/content/Korpora",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls",
    learning_rate=5e-5,
    batch_size=32,
    )

In [ ]:
# 데이터 다운로드

from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
)

# Korpora라는 패키지는 다양한 한국어 말뭉치를 쉽게 내려받고 전처리 할 수 있도록 도와준다.

[nsmc] download ratings_train.txt: 14.6MB [00:00, 90.8MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 35.3MB/s]                           


In [ ]:
# 프리트레인을 마친 모델 준비하기
# kcbert-base모델을 준비하는 코드


from transformers import BertConfig, BertForSequenceClassification
# BertConfig: BERT 모델의 구성(config)을 정의하는 클래스. 모델의 하이퍼파라미터와 구조를 설정
# BertForSequenceClassification: BERT 모델을 텍스트 분류 작업에 맞게 수정한 클래스. 마지막 레이어가 분류 작업에 적합하도록 조정되어 있다.

# 사전 훈련된 BERT 모델의 구성 파일을 로드
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,  # 사전 훈련된 모델의 이름이나 경로를 나타냄
    num_labels=2,                # 분류 작업의 레이블 수를 지정. 이 경우, 두 개의 레이블을 가진 이진 분류 문제를 다루고 있음
)

# 사전 훈련된 BERT 모델을 로드
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,    # args.pretrained_model_name으로부터 모델의 가중치를 불러옴
    config=pretrained_model_config,# 위에서 생성한 구성 객체를 전달하여, 모델을 텍스트 분류 작업에 맞게 설정
)


#  사전 훈련된 BERT 모델을 불러와서, 텍스트 분류 작업에 맞게 설정하고 초기화하는 과정.
#   ㄴ 모델의 하이퍼파라미터와 구성을 정의하고, 이 설정을 바탕으로 BERT 모델을 생성하여 이후의 분류 작업에 사용할 준비

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly i

* 토큰 : 문장보다 작은 단위
* 토큰화 : 문장을 토큰 시퀀스로 분석하는 과정
* 토크나이저 : 토큰화를 수행하는 프로그램

In [ ]:
#토크나이저 준비

from transformers import BertTokenizer       # BertTokenizer : 텍스트 데이터를 모델이 이해할 수 있는 입력 형식으로 변환 (주로 단어를 토큰으로 나누고,토큰을 숫자 ID로 변환하는 기능)
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,              # 전 훈련된 BERT 모델의 이름이나 경로. 이 이름을 통해 관련된 토크나이저의 사전 훈련된 가중치와 설정을 로드.
    do_lower_case=False,                     # 입력 텍스트를 소문자로 변환할지 여부를 결정. BERT의 사전 훈련 모델이 대문자와 소문자 구분을 포함하고 있을 때 이 옵션을 사용
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]